In [13]:
# pip install transformers sentencepiece
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from numpy import dot
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd


tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")


tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
# model.cuda()  # uncomment it if you have a GPU

def embed_bert(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]  # вектор токена [CLS], который является агрегированной информацией о всём тексте
    #embeddings = model_output.pooler_output
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

# print(embed_bert_cls('привет мир', model, tokenizer))
# (312,)

/home/kolya/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [14]:
def cos_distance(e1, e2):
    return 1 - dot(e1, e2)/np.linalg.norm(e1)/np.linalg.norm(e2)

class Retrieval:
    def __init__(self, chunks, model, tokenizer):
        if isinstance(chunks, pd.DataFrame):
            self.chunks = list(chunks["chunks"])
        elif isinstance(chunks, list):
            self.chunks = chunks
        else:
            raise TypeError('чанки должны быть или списком или датафреймом')
        self.model = model
        self.tokenizer = tokenizer
        self.chunk_vecs = []
        for chunk in self.chunks:
            self.chunk_vecs.append(list(embed_bert(chunk, model, tokenizer)))
        self.chunk_vecs = np.array(self.chunk_vecs)

    def find_context(self, question):
        q_emb = embed_bert(question, self.model, self.tokenizer)
        min_ind = 0
        distances = []
        for i in range(len(self.chunks)):
            distances.append(cos_distance(q_emb, self.chunk_vecs[i]))
        min_ind = np.argmin(distances)

        return self.chunks[min_ind], distances[min_ind], min_ind, distances

    def find_top_k_context(self, question, k):
        q_emb = embed_bert(question, self.model, self.tokenizer)
        min_ind = 0
        distances = []
        for i in range(len(self.chunks)):
            distances.append((cos_distance(q_emb, self.chunk_vecs[i]), i))

        distances.sort(reverse = False, key = lambda x: x[0])
        distances = distances[:k]
        indexes = list(map(lambda x: x[1], distances))
        dist = list(map(lambda x: x[0], distances))
        return indexes, dist
        

In [15]:
df = pd.read_csv('chunks.csv')
retr = Retrieval(df, model, tokenizer)
df

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,chunks,questions,answers
0,Кафедра ФН-11 осуществляет подготовку бакалавр...,Какие образовательные профили подготовки реали...,Кафедра ФН-11 реализует следующие образователь...
1,В 1878г. заведующим кафедрой общей и прикладно...,Какие научные достижения были у профессора П.А...,Профессор П.А. Зилов: В 1877 году измерил диэ...
2,Главный учебный корпус (ГУК) МГТУ состоит из д...,Из каких частей состоит главный учебный корпус...,Главный учебный корпус МГТУ состоит из двух ча...
3,Сообщение между подразделениями университета о...,Как устроено сообщение между подразделениями и...,Сообщение между подразделениями университета о...
4,Учебно-лабораторный корпус был открыт 1 марта ...,Когда был открыт учебно-лабораторный корпус (У...,Учебно-лабораторный корпус (УЛК) был открыт 1 ...
...,...,...,...
69,"Общежитие №5 Адрес: 105043, Москва, Измайловск...",Где находится общежитие №5 и как связаться с к...,"\nОбщежитие №5 расположено по адресу: Москва, ..."
70,информация доступна на странице Общежитие №5. ...,Где находится общежитие № 10?\n,"\nМосква, Госпитальный переулок, 4/6."
71,Почтовые отделения Измайловское Адрес отделени...,Какой график работы у почтового отделения в Из...,"\nГрафик работы: с понедельника — 08:00–20:00,..."
72,- 18:00 ВС выходной Банковские отделения Сберб...,Какой график работы у военкомата в Измайлово и...,\nВоенкомат в Измайлово работает с понедельник...


In [16]:
corr_arr = []
for i in range(len(df)):
    print(f'Номер найденного чанка: {retr.find_top_k_context(df.questions.loc[i], 5)[0]}; Номер вопроса соответствующего чанка: {i}')

Номер найденного чанка: [0, 6, 21, 17, 12]; Номер вопроса соответствующего чанка: 0
Номер найденного чанка: [27, 17, 37, 41, 31]; Номер вопроса соответствующего чанка: 1
Номер найденного чанка: [27, 17, 41, 12, 21]; Номер вопроса соответствующего чанка: 2
Номер найденного чанка: [13, 33, 12, 37, 34]; Номер вопроса соответствующего чанка: 3
Номер найденного чанка: [14, 4, 12, 17, 53]; Номер вопроса соответствующего чанка: 4
Номер найденного чанка: [27, 10, 37, 17, 35]; Номер вопроса соответствующего чанка: 5
Номер найденного чанка: [17, 8, 12, 6, 41]; Номер вопроса соответствующего чанка: 6
Номер найденного чанка: [10, 7, 27, 17, 12]; Номер вопроса соответствующего чанка: 7
Номер найденного чанка: [17, 10, 37, 41, 28]; Номер вопроса соответствующего чанка: 8
Номер найденного чанка: [41, 18, 17, 10, 37]; Номер вопроса соответствующего чанка: 9
Номер найденного чанка: [38, 56, 28, 45, 41]; Номер вопроса соответствующего чанка: 10
Номер найденного чанка: [11, 25, 24, 73, 49]; Номер вопроса

In [17]:
def recall_at_k(indices, relevant_index, k):
    """
    Вычисляет Recall@k
    :param indices: отсортированный список индексов (сортировка по убыванию релевантности)
    :param relevant_index: индекс релевантного документа
    :param k: топ-k
    :return: Recall@k
    """
    # Проверяем, сколько релевантных документов в топ-k
    relevant_found = sum(1 for idx in indices[:k] if idx == relevant_index)
    return relevant_found / 1  # Всего один релевантный документ для каждого запроса

def mrr(indices, relevant_index):
    """
    Вычисляет Mean Reciprocal Rank (MRR)
    :param indices: отсортированный список индексов (сортировка по убыванию релевантности)
    :param relevant_index: индекс релевантного документа
    :return: MRR
    """
    # Находим позицию первого релевантного документа
    for rank, idx in enumerate(indices, start=1):
        if idx == relevant_index:
            return 1 / rank
    return 0  # Если релевантный документ не найден

def mr(indices, relevant_index):
    """
    Вычисляет Mean Reciprocal Rank (MRR)
    :param indices: отсортированный список индексов (сортировка по убыванию релевантности)
    :param relevant_index: индекс релевантного документа
    :return: MRR
    """
    # Находим позицию первого релевантного документа
    for rank, idx in enumerate(indices, start=1):
        if idx == relevant_index:
            return rank
    return 0  # Если релевантный документ не найден


def ndcg_at_k(indices, relevant_index, k):
    """
    Вычисляет NDCG@k
    :param indices: отсортированный список индексов (сортировка по убыванию релевантности)
    :param relevant_index: индекс релевантного документа
    :param k: топ-k
    :return: NDCG@k
    """
    # Вычисляем DCG@k
    dcg_k = sum(1 / np.log2(i + 1) for i, idx in enumerate(indices[:k], start=1) if idx == relevant_index)
    
    # Вычисляем IDCG@k (идеальный случай, когда релевантный чанк на первом месте)
    idcg_k = 1 / np.log2(2) if relevant_index in indices[:k] else 0
    
    return dcg_k / idcg_k if idcg_k > 0 else 0


In [18]:
r1 = []
r3 = []
r5 = []
mrr_metrics = []
mean_rank = []
ndcg1 = []
ndcg3 = []
ndcg5 = []

for i in range(len(df)):
    ind = retr.find_top_k_context(df.questions.loc[i], len(df))[0]
    r1.append(recall_at_k(ind, i, 1))
    r3.append(recall_at_k(ind, i, 3))
    r5.append(recall_at_k(ind, i, 5))
    mrr_metrics.append(mrr(ind, i))
    mean_rank.append(mr(ind,i))
    ndcg1.append(ndcg_at_k(ind, i, 1))
    ndcg3.append(ndcg_at_k(ind, i, 3))
    ndcg5.append(ndcg_at_k(ind, i, 5))


# Вычисляем средние значения
avg_r1 = np.mean(r1)
avg_r3 = np.mean(r3)
avg_r5 = np.mean(r5)
avg_mrr = np.mean(mrr_metrics)
avg_mr = np.mean(mean_rank)
avg_ndcg1 = np.mean(ndcg1)
avg_ndcg3 = np.mean(ndcg3)
avg_ndcg5 = np.mean(ndcg5)

# Выводим средние значения
print(f"Average Recall@1: {avg_r1:.3f}")
print(f"Average Recall@3: {avg_r3:.3f}")
print(f"Average Recall@5: {avg_r5:.3f}")
print(f"Average MRR: {avg_mrr:.3f}")
print(f"Average Mean Rank: {avg_mr:.3f}")
print(f"Average NDCG@1: {avg_ndcg1:.3f}")
print(f"Average NDCG@3: {avg_ndcg3:.3f}")
print(f"Average NDCG@5: {avg_ndcg5:.3f}")

Average Recall@1: 0.243
Average Recall@3: 0.392
Average Recall@5: 0.432
Average MRR: 0.366
Average Mean Rank: 14.378
Average NDCG@1: 0.243
Average NDCG@3: 0.335
Average NDCG@5: 0.353


In [9]:
mean_rank

[1,
 1,
 1,
 4,
 6,
 4,
 5,
 1,
 18,
 1,
 25,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 13,
 1,
 1,
 1,
 1,
 2,
 9,
 1,
 1,
 6,
 3,
 1,
 1,
 9,
 2,
 1,
 3,
 3,
 3,
 1,
 1,
 2,
 2,
 21,
 26,
 7,
 7,
 9,
 14,
 1,
 31,
 48,
 29,
 3,
 4,
 1,
 15,
 1,
 13,
 3,
 6,
 16,
 22,
 14,
 9,
 1,
 4,
 1]

In [10]:
test_ind = 58

print(df.questions.loc[test_ind])

Где и когда студентам нужно визировать анкету для оформления загранпаспорта?




In [11]:
print(df.chunks.loc[test_ind])

ЧТ ПТ СБ выходной ВС Третий отдел ГЗ, 2 этаж, кабинет 235. С 11 мая 2010 года студенты всех факультетов визируют анкету для оформления загранпаспорта в комнате 396б (центральная лестница, 3 этаж). График работы: День Время работы Обед ПН 13:30 - 17:00 - ВТ СР ЧТ ПТ СБ выходной ВС Интернеты Выдача логинов и паролей на университетский WiFi и на интернет во 2 общежитии осуществляется на первом этаже ГЗ, в том же помещении, что и типография. Сотрудникам (не студентам) интернеты выдаются на 2 этаже


In [12]:
retr.find_context(df.questions.loc[test_ind])[0]

'После поступления на курс ты получишь доступ к закрытым вакансиям, сможешь подать заявку и пройти собеседование.'

In [47]:
ind = retr.find_top_k_context(input(), 5)[0]

for i in ind:
    print(retr.chunks[i])
    print()

 какой точный размер стипендии?


Да. На некоторых курсах предъявляются особые требования к характеристикам ноутбука. Например, чтобы заняться разработкой на IOS, нужен макбук.

При соответствии критериям получения стипендии, студент программы от VK будет автоматически внесен в список стипендиатов.Если для оформления стипендии что-то потребуется - к студенту обратится напрямую представитель вуза или VK.

Получение стипендии от компании VK не зависит от того, обучается студент в вузе на бюджетной или платной форме обучения.Стипендия суммируется с академической стипендией, начисляемой вузом.

Обычно от четырёх академических часов в неделю без учёта времени на домашние задания. Но нагрузка на каждой программе может быть разной. .

После поступления на курс ты получишь доступ к закрытым вакансиям, сможешь подать заявку и пройти собеседование.



In [25]:
list(df.chunks)

['Кафедра ФН-11 осуществляет подготовку бакалавров и магистров по направлению "Математика и компьютерные науки (02.03.01; 02.04.01), реализуя образовательные профили подготовки:\n- Математическое и компьютерное моделирование,\n- Математическое и компьютерное моделирование в экономике,\n- Суперкомпьютерное моделирование и искусственный интеллект в инженерных задачах,\n- Математическое и компьютерное моделирование в механике композитов.',
 'В 1878г. заведующим кафедрой общей и прикладной физики стал выпускник Московского университета ученик выдающегося русского физика А.Г.Столетова профессор П.А.Зилов. Профессор П.А.Зилов был первым, кто измерил диэлектрические проницаемости некоторых жидких диэлектриков и доказал правильность соотношения между диэлектрической проницаемостью и показателем преломления (1877г.). В 1879г. П.А.Зилов установил зависимость магнитной проницаемости жидкостей от напряженности магнитного поля.  С декабря 1884г. по август 1886г. кафедрой заведовал профессор Н.П.Слу